<h2>Поставленная задача:<h2>

In [1]:
import re

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from gensim.models import FastText
import fasttext

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

<h1>1. Создаем датасет<h1>

In [2]:
# создаем датасет

def clear_text(text):
    """
        функция для очистки текста:
            приведение к нижнему регистру
            замена «ё» на «е»
            замена ссылок на токен «URL»
            замена упоминания пользователя на токен «USER»
            удаление знаков препинания
    """
    text = text.lower().replace("ё", "е")
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'url', text)
    text = re.sub('@[^\s]+', 'user', text)

    text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip()

# считываем данные
n = ['id', 'date', 'name', 'text', 'typr', 'rep', 'rtw', 'faw', 'stcount', 'foll', 'frien', 'listcount']
data_positive = pd.read_csv('data/positive.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])
data_negative = pd.read_csv('data/negative.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])

# формируем датасет
raw_data = np.concatenate([data_positive['text'].values, data_negative['text'].values])
texts = [clear_text(text) for text in raw_data] # чиcтим тексты
labels = np.array([1] * data_positive.shape[0] + [0] * data_negative.shape[0])

# делим датасет на train/test
texts_train, texts_test, y_train, y_test = train_test_split(texts, labels, test_size=0.1, random_state=1)

<h1>2. Обучение<h1>

<h2>2.1 fasttext от gensim<h2>

In [3]:
# приводим датасет к виду, необходимому для обучения модели
dataset_for_ft = [text.split(' ') for text in texts_train]

# строим вокубуляр токенов
ft_gensim_model = FastText(size=100, window=3, min_count=1)
ft_gensim_model.build_vocab(dataset_for_ft)
total_words = ft_gensim_model.corpus_total_words

# обучаем w2v-модель
ft_gensim_model.train(dataset_for_ft, total_words=total_words, epochs=5)

In [5]:
# находим имбеддинги всех текстов

def text2vec(text):

    vecs = []
    for word in text.split(' '):
        try:
            vecs.append(ft_gensim_model.wv[word])  
        except Exception as e:
            pass      
    mean_vec = np.array(vecs).mean(0)

    return mean_vec

x_train = [text2vec(x) for x in texts_train]
x_test = [text2vec(x) for x in texts_test]

Результаты для fasttext от gensim

In [7]:
# обучаем логистическую регрессию
clf_logreg = LogisticRegression(random_state=0, max_iter=200).fit(x_train, y_train)

# смотрим результат на test-выборке
pred = clf_logreg.predict(x_test)
print('logistic regression accuracy on test: {}'.format(1 - np.abs(pred - y_test).sum() / len(y_test)))


# обучаем наивный байес
clf_naive_b = GaussianNB().fit(x_train, y_train)

# смотрим результат на test-выборке
pred = clf_naive_b.predict(x_test)
print('naive bayes accuracy on test: {}'.format(1 - np.abs(pred - y_test).sum() / len(y_test)))

logistic regression accuracy on test: 0.6900899312290601
naive bayes accuracy on test: 0.6326926467995062


<h2>2.2 fasttext от facebook<h2>

In [8]:
# приводим датасет к виду, необходимому для обучения модели

with open('ft_dataset.txt', 'w') as f:
    f.write('\n\n'.join(texts_train))

# обучаем w2v-модель
ft_fb_model = fasttext.train_unsupervised(input="ft_dataset.txt")

In [9]:
# находим имбеддинги всех текстов

def text2vec(text):

    vecs = []
    for word in text.split(' '):
        try:
            vecs.append(ft_fb_model.get_word_vector(word))  
        except Exception as e:
            pass      
    mean_vec = np.array(vecs).mean(0)

    return mean_vec
    
x_train = [text2vec(x) for x in texts_train]
x_test = [text2vec(x) for x in texts_test]

Результаты для fasttext от facebook

In [10]:
# обучаем логистическую регрессию
clf_logreg = LogisticRegression(random_state=0, max_iter=200).fit(x_train, y_train)

# смотрим результат на test-выборке
pred = clf_logreg.predict(x_test)
print('logistic regression accuracy on test: {}'.format(1 - np.abs(pred - y_test).sum() / len(y_test)))


# обучаем наивный байес
clf_naive_b = GaussianNB().fit(x_train, y_train)

# смотрим результат на test-выборке
pred = clf_naive_b.predict(x_test)
print('naive bayes accuracy on test: {}'.format(1 - np.abs(pred - y_test).sum() / len(y_test)))

logistic regression accuracy on test: 0.7240786457414918
naive bayes accuracy on test: 0.685946041262564
